In [1]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
from pandas import DataFrame
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import History 
history = History()

In [2]:
# input the name of dataset
# Example: Kefiran
name_paper = input('input name :')

input name :Kefiran


In [3]:
# import the nomalized of the dataset
address = 'norm_file/norm_%s.csv' %(name_paper)
dataset = pd.read_csv(address)

In [ ]:
# X is the features of the ANN model
# Y is the target of the ANN model
X = dataset.drop(['OD'], axis = 1)
y = dataset['OD']

In [ ]:
# Split the dataset to training and testing data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

# Save the training and tesing dataset in the test_train folder
# Must create the folder before start running the program
X_train.to_csv('test_train/%sX_train.csv' %(name_paper))
X_test.to_csv('test_train/%sX_test.csv' %(name_paper))
y_train.to_csv('test_train/%sy_train.csv' %(name_paper))
y_test.to_csv('test_train/%sy_test.csv' %(name_paper))

In [ ]:
# The function for difind the number of neurons per each hiddinlayer
# And defind the activation function
def num_neuron(n1, n2, n3, act):
    model.add(Dense(n1,input_shape=(4,), activation=act))
    model.add(Dense(n2,activation=act))
    model.add(Dense(n3,activation=act))
    model.add(Dense(1,activation='linear'))

# The function for calculate the error value of the model
# Including MSE, MAE, and PMAE
def find_error(ypred, ytest):
    mse = 0
    mae = 0
    percenterror = 0
    for i in range(0, len((ypred))):
        mse += (ypred[i] - ytest[i])**2
        mae += abs(ypred[i] - ytest[i])
        percenterror += (abs(ypred[i] - ytest[i]))*100/ytest[i]
    mse = mse/len(ypred)
    mae = mae/len(ypred)
    rmse = np.sqrt(mse)
    percenterror = percenterror/len(ypred)
    errorr = [mae, mse, rmse, percenterror]
    return errorr

In [ ]:

nlable = []
errorvalue = []
func = ['relu', 'sigmoid', 'tanh']

for act in func:
    for i in range(4, 31):
        for j in range(4, 31):
            for k in range(4, 31):
                model = Sequential()
                num_neuron(i, j, k, act)
                model.compile(loss='mse',optimizer='adam',metrics=['mape','mse'])
                hist = model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=500)
                train_history = DataFrame(hist.history)
                
                y_pred = model.predict(X_test)
                
                # change y_pred and y_test to list
                y_testlistt=y_test.values.tolist()
                y_predlistt = []
                for l in range(0,len(y_pred)):
                    y_predlistt.append(y_pred[l][0])
            
                # collect error
                errorr = find_error(y_predlistt, y_testlistt)
                errorvalue.append(errorr)
                
                # save the model
                model_name = '%s_%d_%d_%d' %(act, i, j, k) 
                nlable.append(model_name)
            
                address_save = 'last_last_test_%s/%s_%s.h5' %(name_paper, name_paper, model_name)
                model.save(address_save)
                
                #save training history
                address_train = 'last_last_test_%s/hist/%s_hist_%s.csv' %(name_paper, name_paper, model_name)
                train_history.to_csv(address_train)
                
                #save error
                error_table = DataFrame(errorvalue,columns=['mae', 'mse', 'rmse', 'Perror'])
                error_table['neuron'] = nlable
                error_table = error_table[['neuron', 'mae', 'mse', 'rmse', 'Perror']]
                error_tablee = error_table.sort_values(by=['mse'])
                address = 'error_table/last_last_test_%s_error.csv' %(name_paper)
                error_tablee.to_csv(address)